# Download data

In [1]:
import os
import tarfile
from six.moves import urllib

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

In [3]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [4]:
fetch_housing_data()

# Load data & Take a qucik look

In [5]:
import pandas as pd
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [6]:
housing_data = load_housing_data()
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [7]:
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [8]:
%matplotlib inline

In [9]:
import matplotlib.pyplot as plt

# Create a test set

In [10]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing_data, test_size=0.2, random_state=42)

In [11]:
len(train_set), len(test_set)

(16512, 4128)

# Stratified sampling

In [12]:
import numpy as np
housing_data["income_cat"] = np.ceil(housing_data["median_income"]/1.5)
housing_data["income_cat"].where(housing_data["income_cat"] < 5, 5.0, inplace=True)

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing_data, housing_data["income_cat"]):
    strat_train_set = housing_data.loc[train_index]
    strat_test_set = housing_data.loc[test_index]

In [14]:
housing_data["income_cat"].value_counts()/len(housing_data)

3.0    0.350581
2.0    0.318847
4.0    0.176308
5.0    0.114438
1.0    0.039826
Name: income_cat, dtype: float64

In [15]:
strat_train_set["income_cat"].value_counts()/len(strat_train_set)

3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114402
1.0    0.039850
Name: income_cat, dtype: float64

In [16]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Prepare the data for ML

In [17]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [18]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")

In [19]:
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [20]:
imputer.statistics_

array([ -118.51  ,    34.26  ,    29.    ,  2119.5   ,   433.    ,
        1164.    ,   408.    ,     3.5409])

In [21]:
housing_num.median().values

array([ -118.51  ,    34.26  ,    29.    ,  2119.5   ,   433.    ,
        1164.    ,   408.    ,     3.5409])

In [22]:
X = imputer.transform(housing_num)

In [23]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [24]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer(sparse_output=True)
housing_cat = housing["ocean_proximity"].copy()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<type 'numpy.int32'>'
	with 16512 stored elements in Compressed Sparse Row format>

# Custom transformers

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, population_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

# Feature scaling & transformation pipeline

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [27]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
class LabelBinarizerPipelineFriendly(LabelBinarizer):
    """this would allow us to fit the model based on the X input."""
    def fit(self, X, y=None):
        super(LabelBinarizerPipelineFriendly, self).fit(X)
    def transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).transform(X)
    def fit_transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).fit(X).transform(X)

In [28]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', LabelBinarizerPipelineFriendly()),
])

from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)

In [29]:
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ..., 
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

In [30]:
housing_prepared.shape

(16512L, 16L)

# Select and train models (SVM)

In [31]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [33]:
from sklearn.svm import SVR
svm_reg = SVR(kernel='linear', C=1e3)
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
from sklearn.metrics import mean_squared_error
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
print(svm_rmse)
from sklearn.model_selection import cross_val_score
svm_scores = cross_val_score(svm_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
svm_rmse_scores = np.sqrt(-svm_scores)
display_scores(svm_rmse_scores)

70365.5352921
('Scores:', array([ 67644.35384499,  68243.22099454,  72254.90837144,  74688.96841464,
        68878.42423191,  73392.15042354,  66532.13342718,  69741.97528703,
        73484.82122714,  69924.12972421]))
('Mean:', 70478.508594662155)
('Standard deviation:', 2653.7464157277959)


# Fine-tune models

In [38]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'C': [10 ** i for i in range(-2, 4, 1)]},
]

svm_reg = SVR(kernel='linear')

gird_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
gird_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.01, 0.1, 1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [39]:
gird_search.best_params_

{'C': 1000}

In [40]:
gird_search.best_estimator_

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [42]:
cvres = gird_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

(63986.486379360715, {'max_features': 2, 'n_estimators': 3})
(55534.391396345527, {'max_features': 2, 'n_estimators': 10})
(52856.444400999746, {'max_features': 2, 'n_estimators': 30})
(61303.175792038317, {'max_features': 4, 'n_estimators': 3})
(53638.291967153222, {'max_features': 4, 'n_estimators': 10})
(50889.798513048343, {'max_features': 4, 'n_estimators': 30})
(59767.999004834361, {'max_features': 6, 'n_estimators': 3})
(52738.686839464361, {'max_features': 6, 'n_estimators': 10})
(50811.551373317161, {'max_features': 6, 'n_estimators': 30})
(58808.81113236535, {'max_features': 8, 'n_estimators': 3})
(52597.173758265482, {'max_features': 8, 'n_estimators': 10})
(50787.824132513735, {'max_features': 8, 'n_estimators': 30})
(62038.02627041813, {'max_features': 2, 'n_estimators': 3, 'bootstrap': False})
(54164.723018595192, {'max_features': 2, 'n_estimators': 10, 'bootstrap': False})
(60169.162985813782, {'max_features': 3, 'n_estimators': 3, 'bootstrap': False})
(53078.55789793578

# Evaluate on the test set

In [42]:
final_model = gird_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [43]:
final_rmse

68458.597646802518

# RandomizedSearchCV

In [44]:
from sklearn.svm import SVR
svm_reg = SVR(kernel='rbf', C=1e3, gamma=0.1)
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
from sklearn.metrics import mean_squared_error
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
print(svm_rmse)
from sklearn.model_selection import cross_val_score
svm_scores = cross_val_score(svm_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
svm_rmse_scores = np.sqrt(-svm_scores)
display_scores(svm_rmse_scores)

69485.4547821
('Scores:', array([ 67813.98178449,  68890.38265573,  70429.44709768,  72002.05738789,
        69420.37173834,  74097.8041831 ,  66719.82957121,  70394.47002374,
        73189.59870112,  70294.69885874]))
('Mean:', 70325.26420020638)
('Standard deviation:', 2178.2584066591971)


In [47]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'C': [10 ** i for i in range(-5, 5, 1)],
              'gamma': [10 ** i for i in range(-5, 5, 1)]}
n_iter_search = 10

svm_reg = SVR(kernel='rbf')

rand_search = RandomizedSearchCV(svm_reg, param_distributions=param_dist, n_iter=n_iter_search, scoring='neg_mean_squared_error')
rand_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'gamma': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=0)

In [49]:
rand_search.best_estimator_

SVR(C=10000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=10,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [50]:
cvres = rand_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

(118829.13837488796, {'C': 10, 'gamma': 0.001})
(118935.40717386162, {'C': 10, 'gamma': 10000})
(118935.00696569699, {'C': 0.01, 'gamma': 0.0001})
(118935.00693944238, {'C': 0.1, 'gamma': 1e-05})
(118935.05522115828, {'C': 1, 'gamma': 100})
(118935.01836049047, {'C': 0.0001, 'gamma': 1e-05})
(118039.46763173386, {'C': 10, 'gamma': 0.01})
(114585.56698813898, {'C': 10000, 'gamma': 10})
(118935.01837077964, {'C': 1e-05, 'gamma': 1e-05})
(118826.60158594423, {'C': 1000, 'gamma': 1e-05})


# a "fuller" pipeline?

In [51]:
def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]

In [54]:
feature_importances = np.array([  7.33442355e-02,   6.29090705e-02,   4.11437985e-02,
                                  1.46726854e-02,   1.41064835e-02,   1.48742809e-02,
                                  1.42575993e-02,   3.66158981e-01,   5.64191792e-02,
                                  1.08792957e-01,   5.33510773e-02,   1.03114883e-02,
                                  1.64780994e-01,   6.02803867e-05,   1.96041560e-03,
                                  2.85647464e-03])
k=7

prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('svm_reg', SVR(**gird_search.best_params_))
])

In [55]:
prepare_select_and_predict_pipeline.fit(housing, housing_labels)

Pipeline(memory=None,
     steps=[('preparation', FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(attribute_names=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'])), ('... epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])

In [57]:
some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]

print("Predictions:", prepare_select_and_predict_pipeline.predict(some_data))
print("Labels:", list(some_labels))

('Predictions:', array([ 179005.98742851,  342855.78532807,  163900.16922675,
         61064.53586579]))
('Labels:', [286600.0, 340600.0, 196900.0, 46300.0])
